In [1]:
%region us-east-1
%iam_role #setar
%idle_timeout 30
%glue_version 4.0
%worker_type G.1X
%number_of_workers 6

#%additional_python_modules s3://fiocruz-datalake-bucket/raw/aesop/dados_auxiliares/python_utils/teste.py
%extra_py_files s3://fiocruz-datalake-bucket/raw/aesop/dados_auxiliares/python_utils/util_custom.py

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Previous region: None
Setting new region to: us-east-1
Region is set to: us-east-1
Current iam_role is None
iam_role has been set to arn:aws:iam::673245419703:role/service-role/AWSGlueServiceRole.
Current idle_timeout is None minutes.
idle_timeout has been set to 30 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 6
Extra py files to be included:
s3://fiocruz-datalake-bucket/raw/aesop/dados_auxiliares/python_utils/util_custom.py
Trying to create a Glue session for the kernel.
Session Type: etl
Worker Type: G.1X
Number of Workers: 6
Session ID: 79341c26-605e-47

In [2]:
from util_custom import *
from pyspark.sql import functions as F
from pyspark.sql.types import *
import os
from datetime import datetime, timedelta
#from epiweeks import Week

In [8]:
#df = spark.read.parquet(aesop_parquet_path+get_parquet_file_name())
#aesop_parquet_path+get_parquet_file_name()

In [3]:
df = spark.read.parquet("s3://fiocruz-datalake-bucket/raw/aesop/dados-ms-parquet/aesop_dados_2017_20240414.parquet/")

In [4]:
df.count() #1091155616 -> 1095193151 ->  1099462094 -> 

1100855294


In [8]:
#df.show(10)

In [5]:
n_semana_max = get_max_semana_from_aesop_dados(df, 'ano', 'semana')
n_semana_max

14


In [6]:
res = df.groupBy("ano").agg(F.max("semana").alias("max_semana"),F.min("semana").alias("min_semana"))

In [7]:
res.show(100)

+----+----------+----------+
| ano|max_semana|min_semana|
+----+----------+----------+
|2019|        52|         1|
|2017|        52|         1|
|2016|        52|         1|
|2022|        52|         1|
|2018|        52|         1|
|2023|        52|         1|
|2024|        14|         1|
|2021|        52|         1|
|2020|        53|         1|
|null|      null|      null|
+----+----------+----------+


#### gerar data

In [8]:
# Define a UDF to apply the date calculation

def return_day(week, year):
    # Calculate the timestamp for December 31 of the previous year
    prev_year = year - 1
    timestamp_dec_31 = datetime(prev_year, 12, 31)
    # Calculate the target date by adding (week * 7) days and rest 3 days
    target_date = timestamp_dec_31 + timedelta(days=week * 7 - 3)
    return target_date

# Register the UDF
return_day_udf = F.udf(return_day, DateType())

# Apply the UDF to the DataFrame
df = df.withColumn("DATE", return_day_udf(df["SEMANA"], df["ANO"]))

#### carregar arquivos auxiliares

In [9]:
df_v2 = df\
.filter(F.col("ANO") >= 2017)\
.filter(F.col("FX_ETARIA").isNotNull())

#### formata CID-CIAP

In [10]:
#criar variável para o tipo do codigo, pode ser CID/CIAP/AB , usa-se split para quebrar a variável de entrada pelo primeiro paratese 
df_v2 = df_v2.withColumn("tipo_codigo", F.split(F.col("CIDCIAP"), "\\(").getItem(0))
#remover ponto dos cids/ciap/ab
df_v2 = df_v2.withColumn("CIDCIAP_v2", F.regexp_replace(F.col("CIDCIAP"), "\\.", ""))

# extrair codigo de dentro dos parenteses
# exemplo CID(Z00) -> Z00
regex_pattern = r'\(([^)]+)\)'
df_v2 = df_v2.withColumn("CIDCIAP_v3", F.trim(F.regexp_extract(F.col("CIDCIAP_v2"), regex_pattern, 1)))

# quem tiver 4 digitos vai ficar somente com 3 digitos para ser usado na criação da variável ivas posteriomente
# ex  L028 -> L02
condition = (F.length(F.col('CIDCIAP_v3')) == 4) #esse jeito não serve para a nova var de arbovirose
df_v2 = df_v2.withColumn(
    'Codigo_CIDCIAP_3digits',F.when(condition, F.expr("substring(CIDCIAP_v3, 1, length(CIDCIAP_v3)-1)"))
    .otherwise(F.col('CIDCIAP_v3'))
)

# concatena tipo + codigo 
df_v2 = df_v2.withColumn("tipo_codigo_3digits", F.concat_ws("", F.col("tipo_codigo"), F.col("Codigo_CIDCIAP_3digits") ) )
df_v2 = df_v2.withColumn("tipo_codigo_full_digits", F.concat_ws("", F.col("tipo_codigo"), F.col("CIDCIAP_v3") ) )

df_v3 = df_v2.drop("CIDCIAP") # tipo_codigo é o tipo do código pode ser CID/CIAP/AB

#### IVAS

In [11]:
# As IVAS são infecções virais das vias aéreas superiores
codigos_ivas_numero = ['CIAPA03','CIAPR01','CIAPR02','CIAPR03','CIAPR04','CIAPR05','CIAPR07','CIAPR08','CIAPR21','CIAPR23','CIAPR25','CIAPR29','CIAPR71','CIAPR74','CIAPR75',
                       'CIAPR76','CIAPR77','CIAPR78','CIAPR80','CIAPR81','CIAPR83','CIAPR99','CIDJ00','CIDJ01','CIDJ02','CIDJ03','CIDJ04','CIDJ06','CIDJ09','CIDJ10',
                       'CIDJ11','CIDJ12','CIDJ13','CIDJ14','CIDJ15','CIDJ16','CIDJ17','CIDJ18','CIDJ20','CIDJ21','CIDJ22','CIDJ80','CIDR05','CIDR06','CIDR07','CIDR43',
                       'CIDR50','CIDU07','CIDB34','CIDB97']
# cria nova coluna com valores zerados em CIDS/CIAPS fora do ivas
df_v3 = df_v3.withColumn("atend_ivas",F.when( (F.col("tipo_codigo_3digits").isin(codigos_ivas_numero) ), F.col("QT")).otherwise(0))
# zera valores para ivas_exclusao
codigos_ivas_exclusao = ['CIDR072']
df_v4 = df_v3.withColumn("atend_ivas_v2",F.when( (F.col("tipo_codigo_full_digits").isin(codigos_ivas_exclusao) ), 0).otherwise(F.col("atend_ivas")))

#### Arboviroses

In [12]:
# As arboviroses são um grupo de doenças causadas por arbovírus
codigos_arbov_numero = [
    'CIAPA77', 'CIDA90', 'CIDA91', 'CIDA92', 'CIDA920','CIDA925', 'CIDA928', 'CIDA929', 'CIDA99', 'CIDA98',
    'CIDA97','CIDA93','CIDA972','CIDA979','CIDA988','ABABP019'
]
# Dicionário_CódigosARBOV update em 22/12/2023
df_v4 = df_v4.withColumn("atend_arbov",F.when( (F.col("tipo_codigo_full_digits").isin(codigos_arbov_numero) ), F.col("QT")).otherwise(0))#.checkpoint()

In [13]:
df_v5 = df_v4\
.withColumnRenamed('DATE', 'calendar')\
.withColumnRenamed('SEMANA', 'epiweek')\
.withColumnRenamed('ANO', 'epi_year')

In [14]:
# cria base municipal
df_v6 = df_v5.groupBy("CO_MUNICIPIO_IBGE",'epi_year',"epiweek")\
.agg(
    F.sum("QT").alias("atend_totais"),
    F.sum("atend_ivas_v2").alias("atend_ivas"),
    F.last("calendar").alias('calendar'), 
    F.sum("atend_arbov").alias("atend_arbov")
)

In [15]:
df_v7 = df_v6.withColumnRenamed("epi_year","ANO").withColumnRenamed("epiweek","SEMANA")
df_v7 = df_v7.withColumn('Ano_Epiweek', F.substring('calendar', 1,4))
df_v7 = df_v7.withColumn('Mes_Epiweek', F.substring('calendar', 6,2))

#### adiciona cobertura AB

In [16]:
AB_2017_2023_AGGREGATE_FINAL = spark.read.parquet("s3://fiocruz-datalake-bucket/raw/aesop/dados_auxiliares/cobert_ab_17a24.parquet/")
AB_2017_2023_AGGREGATE_FINAL = AB_2017_2023_AGGREGATE_FINAL.withColumnRenamed("Ano","Ano_AB")\
                                                           .withColumnRenamed("Mes","Mes_AB")
AB_2017_2023_AGGREGATE_FINAL = AB_2017_2023_AGGREGATE_FINAL.drop('Unnamed: 0')
AB_2017_2023_AGGREGATE_FINAL = AB_2017_2023_AGGREGATE_FINAL.withColumn('MERGE_COUNT_AB',F.lit(1))

AB_2017_2023_AGGREGATE_FINAL = AB_2017_2023_AGGREGATE_FINAL\
.withColumn("Mes_AB", 
            F.when(F.length(AB_2017_2023_AGGREGATE_FINAL.Mes_AB) < 2,
                   F.concat(F.lit('0'), F.col('Mes_AB'))).otherwise(AB_2017_2023_AGGREGATE_FINAL.Mes_AB)
           )

'''
+------+------+------+---------------+---------------+--------------+
|  Ibge|Ano_AB|Mes_AB|PC_COBERTURA_SF|PC_COBERTURA_AB|MERGE_COUNT_AB|
+------+------+------+---------------+---------------+--------------+
|120001|  2024|    01|            100|            100|             1|
|120005|  2024|    01|            100|            100|             1|
|120010|  2024|    01|            100|            100|             1|
|120013|  2024|    01|            100|            100|             1|
|120017|  2024|    01|            100|            100|             1|
+------+------+------+---------------+---------------+--------------+
'''

df_v8 = df_v7.join(AB_2017_2023_AGGREGATE_FINAL, 
   on=[
     df_v7.CO_MUNICIPIO_IBGE == AB_2017_2023_AGGREGATE_FINAL.Ibge, 
     df_v7.Mes_Epiweek == AB_2017_2023_AGGREGATE_FINAL.Mes_AB,
     df_v7.Ano_Epiweek == AB_2017_2023_AGGREGATE_FINAL.Ano_AB 
   ], how='left')

In [17]:
df_v9 = df_v8.withColumnRenamed("calendar","Date_epiweek")
df_v9 = df_v9.select(
    "CO_MUNICIPIO_IBGE",
    "Date_epiweek",
    "SEMANA",
    "Mes_Epiweek",
    "ANO",
    "atend_totais",
    "atend_ivas",
    "atend_arbov",
    "PC_COBERTURA_SF",
    "PC_COBERTURA_AB"
)

#### add ibge dados

In [18]:
epiweek_ibge2017_2023 = spark.read.csv("s3://fiocruz-datalake-bucket/raw/aesop/dados_auxiliares/epiweek_ibge2017_20241231.csv/",header=True)
#epiweek_ibge2017_2023 = spark.read.parquet("s3://fiocruz-datalake-bucket/raw/aesop/dados_auxiliares/epiweek_ibge2017_20241231.parquet")

schema_epiweek = StructType([
    StructField("semana", StringType(), True),
    StructField("ano", StringType(), True),
    StructField("uf", StringType(), True),
    StructField("co_ibge", StringType(), True),
    StructField("municipio", StringType(), True),
    StructField("pop_2010", StringType(), True),
    StructField("porte", StringType(), True)
])


#epiweek_ibge2017_2023 = spark.read.schema(schema_epiweek).parquet("s3://fiocruz-datalake-bucket/raw/aesop/dados_auxiliares/epiweek_ibge2017_20241231.parquet")

In [19]:
epiweek_ibge2017_2023.count()

2328260


In [20]:
epiweek_ibge2017_2023.printSchema()

root
 |-- semana: string (nullable = true)
 |-- ano: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- co_ibge: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- pop_2010: string (nullable = true)
 |-- porte: string (nullable = true)


In [21]:
epiweek_ibge2017_2023.select("co_ibge").distinct().count()

5570


In [22]:
#epiweek_ibge2017_2023 = spark.read.parquet("s3://fiocruz-datalake-bucket/raw/aesop/dados_auxiliares/epiweek_ibge2017_20241231.parquet/")
# renomear
epiweek_ibge2017_2023 = epiweek_ibge2017_2023\
.withColumnRenamed("uf","uf_ibge")\
.withColumnRenamed("ano","ano_ibge")\
.withColumnRenamed("semana","semana_ibge")
# cast
epiweek_ibge2017_2023 = epiweek_ibge2017_2023\
.withColumn("semana_ibge_int",F.col("semana_ibge").cast(ShortType()))\
.withColumn("ano_ibge",F.col("ano_ibge").cast(ShortType()))\
.withColumn("co_ibge",F.col("co_ibge").cast(IntegerType()))

# anos com 52 semanas
filtro_anos_anteriores = F.col("ano_ibge").isin([2017,2018,2019,2020,2022,2023]) # esses ano não é verificado se tem semana 53. verificar de fato no script de validar e emitir alerta quando surgir
# o ano de 2021 teve 53 semanas
filtro_2021_ano = F.col("ano_ibge") == 2021
filtro_2021_week = F.col("semana_ibge_int") <= 52
# filtrando ano atual
filtro_2024_ano = F.col("ano_ibge") == 2024
filtro_2024_week = F.col("semana_ibge_int") <= n_semana_max
# aplicando filtros
epiweek_ibge2017_2023_filtro = epiweek_ibge2017_2023.filter(filtro_anos_anteriores  | (filtro_2021_ano & filtro_2021_week) | (filtro_2024_ano & filtro_2024_week))

df_v10 = epiweek_ibge2017_2023_filtro.join(df_v9, 
   on=[
     epiweek_ibge2017_2023_filtro.co_ibge == df_v9.CO_MUNICIPIO_IBGE, 
     epiweek_ibge2017_2023_filtro.ano_ibge == df_v9.ANO, 
     epiweek_ibge2017_2023_filtro.semana_ibge_int == df_v9.SEMANA
   ], how='left')

In [23]:
df_v10 = df_v10.withColumn("PC_COBERTURA_SF", F.regexp_replace(F.col("PC_COBERTURA_SF"), ",", ".").cast("double"))
df_v10 = df_v10.withColumn("PC_COBERTURA_AB", F.regexp_replace(F.col("PC_COBERTURA_AB"), ",", ".").cast("double"))

In [24]:
colunas = ['atend_totais', 'atend_ivas', 'atend_arbov', 'PC_COBERTURA_SF', 'PC_COBERTURA_AB']

# Preencher as colunas com zero
fill_values = {coluna: 0 for coluna in colunas}
fill_values

df_v11 = df_v10.fillna(fill_values)

In [25]:
REGIAO_IMEDIATA_BRASIL = spark.read.parquet("s3://fiocruz-datalake-bucket/raw/aesop/dados_auxiliares/Regioes_Imediata_Municípios.parquet/")
REGIAO_IMEDIATA_BRASIL = REGIAO_IMEDIATA_BRASIL.withColumn("CD_GEOCODI_6", F.substring(F.col("CD_GEOCODI"),1,6))

df_v12 = df_v11.join(REGIAO_IMEDIATA_BRASIL, F.col("co_ibge") == F.col("CD_GEOCODI_6"),'left').drop("ANO")

df_v12 = df_v12\
.withColumnRenamed("uf_ibge","UF")\
.withColumnRenamed("ano_ibge","ano") \
.withColumnRenamed("semana_ibge","epiweek")\
.drop("Mes_Epiweek",'UF','CD_GEOCODI')

df_v12 = df_v12.withColumnRenamed("PC_COBERTURA_SF","pc_cobertura_sf")
df_v12 = df_v12.withColumnRenamed("PC_COBERTURA_AB","pc_cobertura_ab")

df_v12 = df_v12.withColumn("epiweek",F.col("epiweek").cast(IntegerType()))

In [26]:
df_legada = df_v12.select(
    "municipio",
    'co_ibge',
    "ano",
    "epiweek",
    "atend_totais",
    "atend_ivas",
    "atend_arbov",
    "pc_cobertura_sf",
    "pc_cobertura_ab",
    "cod_rgimediata",
    "nome_rgi",
    "cod_rgint",
    "nome_rgint"
) 

#### Salvar arquivo

In [9]:
#diretorio = aesop_hpc_path+"2017_"+str(get_latest_date_aesop())+"_AESOP.parquet"
#diretorio

In [28]:
df_legada.write.parquet("s3://fiocruz-datalake-bucket/standard/output_hpc/2017_20240414_AESOP.parquet")

In [29]:
single_file = spark.read.parquet("s3://fiocruz-datalake-bucket/standard/output_hpc/2017_20240414_AESOP.parquet")

In [30]:
single_file.count() #2094320 ->2099890 ->2105460 ->

2111030


In [31]:
single_file\
.repartition(1)\
.write.format("parquet").mode("error")\
.save("s3://fiocruz-datalake-bucket/standard/output_hpc_single_file/2017_20240414_AESOP.parquet", mode = 'overwrite')

In [49]:
#renomear_arquivo_unico(diretorio, "base_aps_atual.parquet")

In [11]:
%stop_session

Stopping session: 79341c26-605e-470b-a235-94928d478cdb
Stopped session.
